In [1]:
import numpy as np
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumCircuit, ParameterVector, Parameter
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, ADAM
from qiskit.opflow import MatrixOp
import warnings
from qiskit.circuit.library import RZGate, RXGate, RXXGate
from math import pi

warnings.filterwarnings('ignore')


import qutip as qt

/var/folders/k_/0jg_dk8x3519whfrw207hfzr0000gn/T/ipykernel_94165/1896842105.py:5: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import VQE, NumPyMinimumEigensolver


In [2]:
sx = qt.sigmax()
sy = qt.sigmay()
sz = qt.sigmaz()
si = qt.qeye(2)

def find_neighbors(site, L=2):
    row, col = divmod(site, L)
    nearest_neighbors = []
    next_nearest_neighbors = []

    # Nearest neighbors
    if row > 0: nearest_neighbors.append((row - 1) * L + col)  # Up
    if row < L - 1: nearest_neighbors.append((row + 1) * L + col)  # Down
    if col > 0: nearest_neighbors.append(row * L + (col - 1))  # Left
    if col < L - 1: nearest_neighbors.append(row * L + (col + 1))  # Right

    # Next-nearest neighbors
    if row > 0 and col > 0: next_nearest_neighbors.append((row - 1) * L + (col - 1))  # Up-Left
    if row > 0 and col < L - 1: next_nearest_neighbors.append((row - 1) * L + (col + 1))  # Up-Right
    if row < L - 1 and col > 0: next_nearest_neighbors.append((row + 1) * L + (col - 1))  # Down-Left
    if row < L - 1 and col < L - 1: next_nearest_neighbors.append((row + 1) * L + (col + 1))  # Down-Right

    return nearest_neighbors, next_nearest_neighbors

# Define the kronecker product (tensor product) function for QuTiP objects
def tensor_product_qutip(ops):
    return qt.tensor(ops)

# Function to create site operators in the many-body Hilbert space
def site_operator(N, op, site):
    op_list = [si] * N
    op_list[site] = op
    return tensor_product_qutip(op_list)

# Hamiltonian construction function for the 2x2 lattice
# Hamiltonian construction function for the 2x2 lattice
def construct_hamiltonian_qutip(L=2, J2=0.5):
    N_sites = L * L  # Total number of sites
    H = 0  # Start with an empty Hamiltonian
    
    for site in range(N_sites):
        # Add nearest-neighbor interactions
        nearest_neighbors, _ = find_neighbors(site, L=L)
        for neighbor in nearest_neighbors:
            if site < neighbor:  # To avoid double counting
                H += (1/4) * (site_operator(N_sites, sx, site) * site_operator(N_sites, sx, neighbor) +
                               site_operator(N_sites, sy, site) * site_operator(N_sites, sy, neighbor) +
                               site_operator(N_sites, sz, site) * site_operator(N_sites, sz, neighbor))
        # Add next-nearest-neighbor interactions
        _, next_nearest_neighbors = find_neighbors(site, L=L)
        for neighbor in next_nearest_neighbors:
            if site < neighbor:  # To avoid double counting
                H += J2 * (site_operator(N_sites, sx, site) * site_operator(N_sites, sx, neighbor) +
                            site_operator(N_sites, sy, site) * site_operator(N_sites, sy, neighbor) +
                            site_operator(N_sites, sz, site) * site_operator(N_sites, sz, neighbor))
    
    return H


# Construct the Hamiltonian for a 2x2 lattice
H_qutip = construct_hamiltonian_qutip(L=2, J2=0.5)
H_qutip

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.5  0.   0.5  0.   0.   0.   1.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.5  0.   0.   1.   0.   0.   0.   0.5  0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.  -1.   0.   1.   0.5  0.   0.   0.5  1.   0.   0.   0.
   0.   0. ]
 [ 0.   0.5  1.   0.   0.   0.   0.   0.   0.5  0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   1.   0.  -1.   0.5  0.   0.   0.5  0.   0.   1.   0.
   0.   0. ]
 [ 0.   0.   0.   0.5  0.   0.5  0.   0.   0.   0.   0.5  0.   0.5  0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.5  0.   0.5
   1.   0. ]
 [ 0.   1.   0.5  0.   0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.5  0.   0.5  0.   0.   0.   0.   0.5  0.   0.5  0.
   0.   0. ]
 [ 0.   0.   0.   1.   0.   0.   0.5  0.   0.   0

In [3]:
H_np = H_qutip.full()

In [4]:

len(H_np)

16

In [5]:
hamiltonian_matrix= H_np
hamiltonian_operator = MatrixOp(hamiltonian_matrix)

def create_ansatz(num_qubits, depth):
    # Assume the depth and the specific gates are determined from the image
    depth = 4  # For example, change according to the layers in the image
    ansatz = QuantumCircuit(num_qubits)

    # The rotation angles are parameterized
    parameters = {f"θ_{i}": Parameter(f"θ_{i}") for i in range(depth*num_qubits*3)}

    # Apply Hadamard gates to all qubits
    ansatz.h(range(num_qubits))

    # Apply the layer structure as seen in the image
    for d in range(depth):
        # Insert the parameterized single-qubit rotations and entangling blocks as per the image
        
        # Add layers of parameterized rotation gates (assuming rz(rx(rz)) pattern)
        for qubit in range(num_qubits):
            ansatz.rz(parameters[f"θ_{3*qubit + 3*num_qubits*d}"], qubit)
            ansatz.rx(parameters[f"θ_{3*qubit + 1 + 3*num_qubits*d}"], qubit)
            ansatz.rz(parameters[f"θ_{3*qubit + 2 + 3*num_qubits*d}"], qubit)

        # Add entangling gates based on the pattern in the circuit diagram
        # This is a simplification and may need adjustments to match the specific pattern of entanglements
        for qubit in range(0, num_qubits - 1, 2):
            ansatz.cx(qubit, qubit + 1)
        for qubit in range(1, num_qubits - 1, 2):
            ansatz.cx(qubit, qubit + 1)
            
        # Add more layers if there are additional patterns in the image

    return ansatz, list(parameters.values())



num_qubits = int(np.log2(hamiltonian_matrix.shape[0]))
ansatz, parameters = create_ansatz(num_qubits,1)

# Use SPSA optimizer, it's suitable for noisy optimization like on a real quantum device
optimizer = ADAM(maxiter=1000)

# Setup quantum instance to use the statevector simulator
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))

# Initialize VQE with the ansatz, optimizer, and the quantum instance
vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

# Run VQE to find the lowest eigenvalue of the Hamiltonian
vqe_result = vqe.compute_minimum_eigenvalue(operator=hamiltonian_operator)

# Extract the lowest eigenvaluef
lowest_eigenvalue = np.real(vqe_result.eigenvalue)
print("The VQE given lowest eigenvalue is: ", lowest_eigenvalue)

# Compare to exact solver
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(operator=hamiltonian_operator)

print('Exact Solver Result:', exact_result.eigenvalue.real)

The VQE given lowest eigenvalue is:  -2.9955311197203995
Exact Solver Result: -2.9999999999999964


In [6]:
import scipy.linalg as la

def find_lowest_eigenvalue(matrix):
    # Compute all eigenvalues, but only the first eigenvectors
    eigenvalues, _ = la.eigh(matrix, eigvals=(0, 0))
    return eigenvalues[0]

# Assuming large_matrix is your matrix
lowest_eigenvalue = find_lowest_eigenvalue(H_np)
print(lowest_eigenvalue)


-3.0
